In [1]:
import BEEtag as BT

import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
import cv2 as cv

# %qtconsole
%matplotlib qt4

In [3]:
print('testing now')

# im = np.array(Image.open('./data/scaleExample.png'))
# im = np.array(Image.open('./data/scaleExample2.png'))

cap = cv.VideoCapture('/Users/nickgravish/Dropbox/Harvard/HighThroughputExpt/Bee_experiments_2016/2016-08-15_13.05.57/5_08-15-16_13-06-13.788_Mon_Aug_15_13-05-57.154_2.mp4')

im = []
for kk in range(100):
    tru, im = cap.read(1)

    im = np.array(im)

    a = BT.BEEtag(visualize=0, localized_threshold_params=(101,0))

    a.set_image(im)
    a.threshold_image()
    a.find_valid_regions()
    a.find_square_regions()

    a.transform_to_grid()
#     a.draw_undistorted()

    a.get_codes()
    print(a.tag_id)
    plt.figure(1)
    a.draw_quad_regions()
    plt.waitforbuttonpress()

# #
# # drawing stuff
# plt.figure(1)
# plt.imshow(a.im)

# plt.figure(2)
# plt.imshow(a.im_gray)

# plt.figure(3)
# plt.imshow(a.BW_Label)

# plt.figure(4)
# for kk, (reg, x, y) in enumerate(zip(a.regions, a.contour_x_points, a.contour_y_points)):
#     plt.subplot(len(a.regions) / 2 + 1, 2, kk+1)
#     plt.imshow(reg.filled_image, interpolation='nearest')
#     plt.title(len(x))
#     plt.plot(x, y, 'o-')

# a.draw_possible_regions()

# a.draw_quad_regions()





testing now
[[], [], []]


/Users/nickgravish/anaconda/lib/python3.4/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


[[], [], []]
[[], []]
[[], [], []]
[[], [], []]
[[], [], []]
[[], [], []]
[[], [], []]
[[], [], []]
[[], [], [], []]
[[], [], []]
[[], []]
[[], [], []]
[[], [], []]
[[], [], [], [], []]
[[], [], [], []]
[[], [], [], [], [], []]


KeyboardInterrupt: 

In [21]:
contour_x_points  = []
contour_y_points = []
square_x_points = []
square_y_points = []
square_regions = []
number_squares = 0

length_epsilon = 10
parallel_epsilon = 15


def unit_vector(vector):
    """
    Returns the unit vector of the vector.
    """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """
    Returns the angle in radians between vectors 'v1' and 'v2'::
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * 180 / np.pi

for kk, reg in enumerate(a.regions):

    # pull out the contours
    imout, contours, hierarchy = cv.findContours(np.uint8(reg.filled_image),
                                   cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

    # Best approximation of a polygon for the shape (not sure about 0 here need to chck TODO)
    tmp = cv.approxPolyDP(contours[0], 0.1 * cv.arcLength(contours[0], True), True)

    x = [pt[0][0] for pt in tmp]
    y = [pt[0][1] for pt in tmp]

    contour_x_points.append(x)  # convert out of opencv's annoying nested list
    contour_y_points.append(y)

    # 4 points in a square
    if len(x) != 4:
        continue
    
    plt.figure(1)
    plt.clf()
    plt.imshow(reg.filled_image)
    plt.draw()
    plt.waitforbuttonpress()
    

    print(kk)
    
    # check if square
    # if square these points will be parallel, and all the same length approximately
    # for x, y in zip(self.contour_x_points[-1], self.contour_y_points[-1]):

    # first check lengths
    lengths = [np.sqrt((x[k % 4] - x[(k + 1) % 4]) ** 2 + (y[k % 4] - y[(k + 1) % 4]) ** 2) for k in np.arange(1, 5)]
    
    error = lengths - np.mean(lengths)
    print(lengths)
    print(error)
    
    OK_lengths = all([err < length_epsilon for err in error])

    # next check paralelism
    dots = [angle_between(np.array([x[k+1], y[k+1]]) - np.array([x[k], y[k]]), 
                          np.array([x[(k+2) % 4], y[(k+2) % 4]])-np.array([x[(k+2+1) % 4], y[(k+2+1) % 4]])) 
                          for k in range(2)]
    print(dots)
    
    OK_parallel = all([dot < parallel_epsilon for dot in dots])

    if OK_lengths and OK_parallel:

        square_x_points.append(x)  # convert out of opencv's annoying nested list
        square_y_points.append(y)

        square_regions.append(reg)

        number_squares += 1

/Users/nickgravish/anaconda/lib/python3.4/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


5
[39.812058474788763, 36.249137920783717, 46.518813398452032, 36.61966684720111]
[ 0.01213931 -3.55078124  6.71889424 -3.18025231]
[10.548065418142151, 0.18562282631851018]
18
[41.868842830916641, 38.118237105091836, 25.632011235952593, 22.825424421026653]
[ 9.75771393  6.00710821 -6.47911766 -9.28570448]
[24.296805284971786, 19.599954405335851]
21
[15.524174696260024, 19.313207915827967, 21.095023109728988, 25.317977802344327]
[-4.78842118 -0.99938797  0.78242723  5.00538192]
[12.160228586310987, 16.498148436700962]


In [3]:
def checkOrs25(imc):
    """
    Checks for valid 5x5 code pattern in each of the 4 possible configurations
    Returns a code and the orientation
    """

    check = []
    codes = []

    # checks all 4 possible orientations of the tag.
    # checks the checksum column and stores the value of whether valid tag or not in in check list
    for cc in range(4):
        imcr = np.rot90(imc, cc)
        check.append(checkCodes25(imcr)) #TODO must return bool
        codes.append(np.reshape(imcr.T, (1, 25))) #TODO may be transpose

    check = np.array(check)

    # there can only be one valid orientation of a correct tag, so there should only be 1 entry that is
    # non zero in the check array. If the sum is 0 or greater than 1 we know it is invalid
    passBin = 0
    if sum(check) == 1:
        passBin = 1

    codesFinal = codes[np.where(check)[0]]
    orienation = np.where(check)[0]

    return codesFinal, orienation

def checkCodes25(imc):
    """
    Checks that code is valid by checking error bits in last 2 columns of 5x5 pattern. Error bits are as follows

    column 4: row bits 1-3 are parity of columns 1-3, row bit 4 is parity of top 3 rows in 3 columns of code.
    (parity of upper left 3x3 code matrix). row bit 5 is parity of bottom 2 rows of 3 columns (bottom left
    2x3 code matrix).

    column 5: reverse of column 4.

    column 5 must equal reverse of column 4 for pass
    """

    im = imc[:, 0:3]
    check_column1 = imc[:, 3].tolist()
    check_column2 = imc[:, 4].tolist()
    check_column2.reverse()

    for first_three_bits in range(3):
        if (np.sum(im[:, first_three_bits]) % 2) != check_column1[first_three_bits]:
            return False
            

    if (np.sum(im[:3,:]) % 2) != check_column1[3]:
        return False

    if (np.sum(im[3:,:]) % 2) != check_column1[4]:
        return False

    if any([c1 != c2 for c1, c2 in zip(check_column1, check_column2)]):
        return False

    # if we got thorough all the checks, return true
    return True




code_locations = np.array([1.5/7, 2.5/7, 3.5/7, 4.5/7, 5.5/7])*a.sc
y_coord, x_coord = np.meshgrid(code_locations, code_locations)
x_coord = np.reshape(x_coord, 25)
y_coord = np.reshape(y_coord, 25)

averaging_width = round((a.sc/7) / 4)

codes = []
valid = []
for code_im in a.undistorted_bw_im:
    tmp_code = [] #np.zeros((5,5), dtype = 'bool')

    for x,y in zip(x_coord, y_coord):
        xrange = np.int32(round(x) + np.array([-averaging_width, averaging_width]))
        yrange = np.int32(round(y) + np.array([-averaging_width, averaging_width]))
        
        tmp_code.append(np.mean(code_im[yrange[0]:yrange[1], xrange[0]:xrange[1]]) > .5)
        
        print(tmp_code[-1])
        
        plt.clf()
        a.draw_undistorted(color=False)
#         plt.gca().invert_yaxis()
        plt.plot(x,y,'ow')
        plt.draw()
        plt.waitforbuttonpress()

#     tmp_code = np.array(tmp_code)
#     tmp_code = np.reshape(tmp_code.T, (5,5))

    tmp_code = np.reshape(tmp_code, (5,5)).T
    # do the heavy lifting to check the code
    final_code, valid_code = checkOrs25(tmp_code)
    codes.append(final_code)
    valid.append(valid_code)

a.id_codes = codes




False


/Users/nickgravish/anaconda/lib/python3.4/site-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


False
False
True
False
True
False
False
False
True
True
False
True
True
True
True
False
False
True
False
False
True
False
False
True


/Users/nickgravish/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:25: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [18]:
def code_to_id(code):
    """
    reshape so that is a 1D array columns first
    """
    code = np.uint8(np.ravel(code.T))
    return int(str().join(str(x) for x in code[0:15]), 2)